# Assignment 4
Logistic Classifiers - Part 2

Group:
* Paola Katherine Pacheco (Facilitator)
* Wai Ping Kwok (Evaluator)

### Problem 2
#### a) [1 mark] Imagine a movie production company wants to use a sentiment analysis model to identify positive/negative reviews of their movies. Which is worse for this use case, a false positive or a false negative, or are they equally bad? What value of β would be suitable for an Fβ score?
ans.

In [1]:
!pip install turicreate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 8.7 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 10.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 7.7 MB/s eta 0:00:0000:01m00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 6.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.4/322.4 kB 6.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 6.6 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 5.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.2/397.2 kB 6.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# import libraries
import turicreate as tc
import numpy as np
import pandas as pd
import random
import seaborn as sns
import matplotlib.pyplot as plt
import re

### Problem 2
#### b) [6 marks] Clean the data so that it is better suited for sentiment analysis, according to your ideas in problem 2h. Add a markdown cell to explain what you are doing.
#### Choose one of two ways to prepare the data:
##### i) Make of copy of the IMBD_Dataset.csv and name it IMBD_Dataset_mod.csv. Open the file in Jupyter Hub and use “find and replace” to clean the text data. OR
##### ii) Load the original dataset into a dataframe and use the regex Python library to clean the text data.
ans.

In [3]:
# load the original dataset into a pandas dataframe
imdb_mod = pd.read_csv("IMDB_Dataset.csv")

# print the dataframe to take a look
imdb_mod

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


### Problem 2
#### c) [2 marks] Load the cleaned data and labels into an SFrame. Add a column named ‘words’ to the SFrame that stores the count of each word used in each review. Print the SFrame.
ans.

In [22]:
! pip install stop-words

  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=d0d94bc853e7d051a4090dc4abfe597a9f45c3b9afc734662a1c400aa084131b
  Stored in directory: /Users/paolakatherine/Library/Caches/pip/wheels/eb/03/0d/3bd31c983789aeb0b4d5e2ca48590288d9db1586cf5f225062
Successfully built stop-words


In [84]:
from stop_words import get_stop_words

all_stop_words = get_stop_words('en')
all_stop_words.remove('not')
all_stop_words.remove('cannot')
all_stop_words.remove('didn\'t')
all_stop_words.append('<br />')

print(all_stop_words)

ValueError: list.remove(x): x not in list

In [87]:
for i in range(len(imdb_mod)):
    text = imdb_mod['review'][i]
    filtered = ' '.join([word for word in text.split() if word not in all_stop_words])
    imdb_mod['filtered_review'][i] = filtered


In [89]:
imdb_mod.tail()

,review,sentiment,filtered_review
49995,I thought this movie did a down right good job...,positive,I thought movie right good job. It creative or...
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,"Bad plot, bad dialogue, bad acting, idiotic di..."
49997,I am a Catholic taught in parochial elementary...,negative,I Catholic taught parochial elementary schools...
49998,I'm going to have to disagree with the previou...,negative,I'm going disagree previous comment side Malti...
49999,No one expects the Star Trek movies to be high...,negative,"No one expects Star Trek movies high art, fans..."


In [128]:
# convert imdb_mod to a SFrame in order to use the count_words 
df_reviews = tc.SFrame(imdb_mod)
df_reviews


df_reviews['words'] = tc.text_analytics.count_words(df_reviews['filtered_review'])
df_reviews

review,sentiment,filtered_review,words
One of the otherreviewers has mentioned ...,positive,One reviewers mentionedwatching just 1 Oz ...,"{'darker': 1.0,'viewing': 1.0, ..."
A wonderful littleproduction. <br /><br ...,positive,A wonderful littleproduction. <br /><br ...,"{'terribly': 1.0,'surface': 1.0, ..."
I thought this was awonderful way to spend ...,positive,I thought wonderful wayspend time hot summer ...,"{'see': 1.0, 'superman':1.0, 'interesting': 1.0, ..."
Basically there's afamily where a little ...,negative,Basically family littleboy (Jake) thinks zombie ...,"{'them': 1.0, 'ignore':1.0, 'dialogs': 1.0, ..."
"Petter Mattei's ""Love inthe Time of Money"" is a ...",positive,"Petter Mattei's ""LoveTime Money"" visually ...","{'work': 1.0,'anxiously': 1.0, ..."
"Probably my all-timefavorite movie, a story ...",positive,"Probably all-timefavorite movie, story ...","{'dozen': 1.0, 'i': 1.0,'if': 1.0, 'startling': ..."
I sure would like to seea resurrection of a up ...,positive,I sure like seeresurrection dated ...,"{'it': 1.0, 'doubt': 1.0,'must': 1.0, 'leave': ..."
"This show was an amazing,fresh & innovative idea ...",negative,"This show amazing, fresh& innovative idea 70's ...","{'awful': 1.0, 'now':1.0, 'as': 1.0, ..."
Encouraged by thepositive comments about ...,negative,Encouraged positivecomments film I looking ...,"{'obsessives': 1.0,'effort': 1.0, 'bit': ..."
If you like original gutwrenching laughter you ...,positive,If like original gutwrenching laughter will ...,"{'br': 2.0, 'mom': 1.0,'hell': 1.0, 'great': ..."


### Problem 2
#### d) [1 mark] Split the data into training/validation/testing sets using 80%/10%/10% respectively.
ans.

In [91]:
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 23.2 MB/s eta 0:00:0000:0100:01


In [115]:
from sklearn.model_selection import train_test_split

# 80% 
train_data, rest_data = train_test_split(df_reviews, train_size=0.8)

# split rest of data in 50%  ( 10% validation + 10% test)
validation_data, test_data = train_test_split(rest_data, test_size=0.5)

print("Train data length:", len(train_data))
print("Validation Data length:", len(validation_data))
print("Test Data length:", len(test_data))

TypeError: Invalid key type: must be str, unicode, bytes or type

In [ ]:
model = tc.logistic_classifier.create(validation_data, features=['words'], target='sentiment')

### Problem 2
#### e) [3 marks] Use Turicreate to create logistic classifiers for sentiment analysis. Be sure to use your validation set in the training. Experiment with different values of hyperparameters to develop two different models.
ans.

### Problem 2
#### f) [8 marks] For each model:
##### i) find predictions using the testing set;
##### ii) display the training/validation/testing accuracy;
##### iii) display the confusion matrix on the testing set;
##### iv) calculate recall, precision, sensitivity and specificity on the testing set;
##### v) calculate the Fβ score on the testing set using the value of β you chose above.
##### vi) using the testing set, calculate points for and plot the ROC curve (use turicreate.evaluation.roc_curve() and then matplotlib.pyplot());
##### vii) calculate the AUC for the testing set.
ans.

### Problem 2
#### g) [2 marks] Select which of your two models is the best (or declare a tie) and justify your choice.
ans.

# Evaluation
<table> 
  <tr>
    <th>Student</th>
    <th>Paola Katherine Pacheco</th>
    <th>Wai Ping Kwok</th>
  </tr>
  <tr>
    <th>Every member contributed</th>
    <th>3</th>
    <th>3</th>
  </tr>
  <tr>
    <th>Every member treated with respect</th>
    <th>3</th>
    <th>3</th>
  </tr>